In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
#import pandas as ps
%matplotlib inline

## Loading and displaying images

load an image  - using OpenCV 

display image - using matplotlib


In [ ]:
image = cv2.imread("games.jpg")
plt.imshow(image) 
print type(image)
print "Number of array dimensions: " + str(image.ndim)
print "Size of image: " + str(image.shape)
print "Number of pixels: " + str(image.size)

**Color image loaded by OpenCV is in BGR mode. But Matplotlib displays in RGB mode. So color images will not be displayed correctly in Matplotlib if image is read with OpenCV. **

In [ ]:
cv2.imshow("Games",image) # a new window will open - close it to continue.
cv2.waitKey(0)
cv2.destroyAllWindows()

To display images loaded by OpenCV - rearrange the color channels.

In [ ]:
b,g,r = cv2.split(image)
img = cv2.merge((r,g,b))
plt.imshow(img)

In [ ]:
fig = plt.figure()
a=fig.add_subplot(1,3,1)
imgplot = plt.imshow(image[:,:,0], cmap = 'Reds')
a.set_title('Red channel')
plt.colorbar(ticks=[0, 255], orientation ='horizontal')
a.set_axis_off()

a=fig.add_subplot(1,3,2)
imgplot = plt.imshow(image[:,:,1], cmap = 'Greens')
a.set_title('Green Channel')
plt.colorbar(ticks=[0, 255], orientation='horizontal')
a.set_axis_off()

a=fig.add_subplot(1,3,3)
imgplot = plt.imshow(image[:,:,2], cmap = 'Blues')
a.set_title('Blue Channel')
plt.colorbar( ticks=[0, 255], orientation='horizontal')
a.set_axis_off()

Leave only the first channel (place zeros in the two others).
See the results in Matplotlib and OpenCV.

In [ ]:
b,g,r = cv2.split(image)
blueImg = image
blueImg[:,:,1] = 0
blueImg[:,:,2] = 0

plt.imshow(blueImg)

plt.figure()
plt.imshow(b, cmap='gray')
plt.colorbar( ticks=[0, 255], orientation='horizontal')

cv2.imshow("Blue?",blueImg)
cv2.waitKey(0)
cv2.destroyAllWindows()


# color conversion

    For color conversion, we use the function 
        cv2.cvtColor(input_image, flag) 
    where flag determines the type of conversion.

In [ ]:
grayImg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

print type(grayImg)
print grayImg.ndim
print grayImg.shape

plt.imshow(grayImg, cmap = 'gray')
plt.colorbar( ticks=[0, 255])

In HSV, it is more easier to represent a color than RGB color-space. 

**Note:** 
For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255]. Different softwares use different scales. So if you are comparing OpenCV values with them, you need to normalize these ranges.

In [ ]:
img = cv2.imread("golf.png")
# Convert BGR to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


fig = plt.figure()
a=fig.add_subplot(1,3,1)
plt.imshow(hsv[:,:,0],cmap='hsv')
plt.xticks([]), plt.yticks([])   # to hide tick values on X and Y axis
plt.title('H')

a = fig.add_subplot(1,3,2)
plt.imshow(img)
plt.xticks([]), plt.yticks([])   # to hide tick values on X and Y axis
plt.title('BGR')

b,g,r = cv2.split(img)       # get b,g,r
rgb_img = cv2.merge([r,g,b])     # switch it to rgb

a = fig.add_subplot(1,3,3)
plt.imshow(rgb_img)
plt.xticks([]), plt.yticks([])   # to hide tick values on X and Y axis
plt.title('RGB')

A simple OpenCV gui, for displaying colors. (press 'esc' to quit).

In [ ]:
def nothing(x):
    pass

# Create a black image, a window
img = np.zeros((300,512,3), np.uint8)
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('R','image',0,255,nothing)
cv2.createTrackbar('G','image',0,255,nothing)
cv2.createTrackbar('B','image',0,255,nothing)

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'image',0,1,nothing)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    # get current positions of four trackbars
    r = cv2.getTrackbarPos('R','image')
    g = cv2.getTrackbarPos('G','image')
    b = cv2.getTrackbarPos('B','image')
    s = cv2.getTrackbarPos(switch,'image')

    if s == 0:
        img[:] = 0
    else:
        img[:] = [b,g,r]

cv2.destroyAllWindows()

#  Filtering 

Gaussian filters.
In OpenCV the function getGaussianKernel() produces a **separable filter** 

In [ ]:
ksize = 11
sigma = 5
#ksize = int(2*np.ceil(3*sigma)+1) # this should be the optimum size
h = cv2.getGaussianKernel(ksize, sigma) # first argument - filter size, second - sigma
H = h.T*h

if ksize < 10:
    print H

plt.imshow(H,interpolation='nearest')
#cv2.getDerivKernels
H.shape

#cv2.Laplacian

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')
X = np.arange(-ksize/2+1, ksize/2 + 1, 1)
Y = np.arange(-ksize/2+1, ksize/2 + 1, 1)
X, Y = np.meshgrid(X, Y)

surf = ax.plot_surface(X, Y, H, rstride=1, cstride=1, linewidth=0, antialiased=False, cmap='jet')
ax.set_zlim(0, H.max())

Filtering operation - convolution:

$g[x,y]*f[x,y] = \displaystyle \sum_{k_1 = -\infty }^\infty \sum_{k_2 = -\infty }^\infty g[k_1,k_2]\cdot f[x-k_1,y-k_2]$

In [ ]:

kernel = np.ones((3,3),np.float32)/9
img = np.zeros((10,10), np.float32)
img[5,5] = 1

dst = cv2.filter2D(img,-1,kernel)

#print img
#print dst

plt.subplot(121),plt.imshow(img,interpolation='nearest'),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(dst,interpolation='nearest'),plt.title('Averaging')
plt.xticks([]), plt.yticks([])

In [ ]:
#ksize = 5
sigma = 1
ksize = int(2*np.ceil(3*sigma)+1)
h = cv2.getGaussianKernel(ksize, sigma)
#print h

from scipy import signal
H = signal.convolve2d(h, h.T, boundary='fill', mode='full', fillvalue=0)
#print H
plt.imshow(H,interpolation='nearest')

##Filtering demo 

from: http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html

parameters - kernel size, sigma x, sigma y (if zero -> sigmas are calculated from kernel size)

In [ ]:
image = cv2.imread("games.jpg")

blur_img = cv2.GaussianBlur(image,(5,5),0)
plt.imshow(blur_img) 

sigX = 8
sigY = 8
blur_img = cv2.GaussianBlur(image,(25,25),sigX, sigY)
plt.imshow(blur_img) 


Median filtering


In [ ]:
median = cv2.medianBlur(image,11)

fig = plt.figure()

a=fig.add_subplot(1,2,1)
plt.imshow(image) 

a=fig.add_subplot(1,2,2)
plt.imshow(median) 